In [1]:
import datasets
import transformers
import pandas as pd
import sklearn
import torch
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset
import torchaudio
import torch
from typing import List
from transformers import get_linear_schedule_with_warmup
import time
from torch.optim import AdamW
from transformers.optimization import Adafactor
import torch.nn as nn
from transformers import HubertForSequenceClassification, Wav2Vec2FeatureExtractor
import torchaudio
import torch
import torch.nn as nn

In [2]:
! wget https://huggingface.co/datasets/KELONMYOSA/dusha_emotion_audio/resolve/main/data/train.tar.gz --no-check-certificate

--2023-07-11 19:39:03--  https://huggingface.co/datasets/KELONMYOSA/dusha_emotion_audio/resolve/main/data/train.tar.gz
Resolving huggingface.co (huggingface.co)... 99.84.108.129, 99.84.108.87, 99.84.108.70, ...
Connecting to huggingface.co (huggingface.co)|99.84.108.129|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f9/36/f936cc9a1834dea987df3d43bad12716beb9c022f2f51034d7372be499277925/906fac85c0d247cd5b98de94b2ed75fa7c1e0bba1e0361ae8413018b5eca3934?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27train.tar.gz%3B+filename%3D%22train.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1689363543&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4OTM2MzU0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mOS8zNi9mOTM2Y2M5YTE4MzRkZWE5ODdkZjNkNDNiYWQxMjcxNmJlYjljMDIyZjJmNTEwMzRkNzM3MmJlNDk5Mjc3OTI1LzkwNmZhYzg1YzBkMjQ3Y2Q1Yjk4ZGU5NGIyZWQ

In [ ]:
! tar -xvzf train.tar.gz

In [4]:
device = 'cuda'

In [351]:
text_model = AutoModelForSequenceClassification.from_pretrained('./model/bert.ppt',ignore_mismatched_sizes=True, return_dict=True, num_labels=5).to(device)
tokenizer = BertTokenizerFast.from_pretrained('cointegrated/rubert-tiny2')

In [352]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
audio_model = HubertForSequenceClassification.from_pretrained("xbgoose/hubert-base-speech-emotion-recognition-russian-dusha-finetuned", output_hidden_states=True)
num2emotion = {0: 'neutral', 1: 'angry', 2: 'positive', 3: 'sad', 4: 'other'}

In [353]:
filepath = "train/84088ff0c9766472f1ba79d2e17e622b.wav"

# waveform, sample_rate = torchaudio.load(filepath, normalize=True)
# transform = torchaudio.transforms.Resample(sample_rate, 16000)
# waveform = transform(waveform)

# inputs = feature_extractor(
#         waveform, 
#         sampling_rate=feature_extractor.sampling_rate, 
#         return_tensors="pt",
#         padding=True,
#         max_length=16000 * 10,
#         truncation=True
#     )

# logits = model(inputs['input_values'][0]).logits
# predictions = torch.argmax(logits, dim=-1)
# predicted_emotion = num2emotion[predictions.numpy()[0]]
# print(predicted_emotion)

In [364]:
class Net(nn.Module):
    def __init__(self, text_model, audio_model):
        super(Net, self).__init__()
        self.text_model = text_model
        self.audio_model = audio_model

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(763, 763, bias=True),
            nn.Tanh(),
            nn.Dropout(p=0.1),
            nn.Linear(763, 5, bias=True)
        )

        self.emotion2label = {
            "neutral":0,
            "positive":1,
            "sad":2,
            "angry":3,
            "other":4
        }

    def tokenize_function(self, text):
        tokenized = tokenizer(text, max_length=512, padding='max_length', return_tensors='pt')
        return torch.cat([tokenized['input_ids'], 
                          tokenized['token_type_ids'], 
                          tokenized['attention_mask']], 0)

    def forward(self, text, audio_path):
        
        text_data = self.tokenize_function(text).unsqueeze(dim=0)
        b_input_ids, b_type_ids, b_input_mask,  = text_data[:, 0, :], text_data[:, 1, :], text_data[:, 2, :]
        # print(b_input_ids)
        logits_a = self.text_model(b_input_ids.cuda().long(),
                       b_input_mask.cuda(),
                       b_type_ids.cuda().long(),
                       output_hidden_states=True
                       # labels=one_hot_labels
                      )['hidden_states'][-1][:, :, -1]#[:, 0, :])

        # print(logits_a)

        waveform, sample_rate = torchaudio.load(filepath, normalize=True)
        transform = torchaudio.transforms.Resample(sample_rate, 16000)
        waveform = transform(waveform)
        
        inputs = feature_extractor(
                waveform, 
                sampling_rate=feature_extractor.sampling_rate, 
                return_tensors="pt",
                padding=True,
                max_length=16000 * 10,
                truncation=True
            )
        
        logits_b = self.audio_model(inputs['input_values'][0].cuda())['hidden_states'][-1][:, :, -1]
        # print(logits_b)
        concatenated_vectors = torch.cat([logits_a.cuda(), logits_b.cuda()], 1)
        output = self.classifier(concatenated_vectors)
        return output
        # return concatenated_vectors

In [365]:
model = Net(text_model, audio_model)

In [366]:
model.to(device)

Net(
  (text_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(83828, 312, padding_idx=0)
        (position_embeddings): Embedding(2048, 312)
        (token_type_embeddings): Embedding(2, 312)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-2): 3 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=312, out_features=312, bias=True)
                (key): Linear(in_features=312, out_features=312, bias=True)
                (value): Linear(in_features=312, out_features=312, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=312, out_features=312, bias=T

In [367]:
for name, param in model.named_parameters():
    if 'text_model' in name or 'audio_model' in name:
        param.requires_grad = False
    else:
        print(name)
        param.requires_grad = True

classifier.1.weight
classifier.1.bias
classifier.4.weight
classifier.4.bias


In [375]:
label2emotion = {
    0: "neutral",
    1: "positive",
    2: "sad",
    3: "angry",
    4: "other"
}
emotion2label = {
            "neutral":0,
            "positive":1,
            "sad":2,
            "angry":3,
            "other":4
        }

In [369]:
train_dataset_dataframe = pd.read_csv('train_dataset.csv')
train_text_frame = pd.read_csv('train_with_text.csv')
train_text_frame.rename(columns = {'audio_path':'file'}, inplace=True)
train_text_frame.drop(columns=['Unnamed: 0'], inplace=True)
train_dataset_dataframe.rename(columns = {'audio_path':'file'}, inplace=True)
train_frame = train_text_frame.merge(train_dataset_dataframe, how='left', on='file')

In [370]:
train_frame

,file,predicted_text,label
0,train/56034f672277aa3aa4c0d5fec976e25f.wav,ты сейчас не говорила,neutral
1,train/84088ff0c9766472f1ba79d2e17e622b.wav,воровайки первый концерт,neutral
2,train/3cf659c3c384271cb22c45297f0f0693.wav,спасибо приятно получать комплимент,neutral
3,train/a9308e85c5cdd1731ed395cef1a226ed.wav,что находится,neutral
4,train/d3bd2f6258e44b2c1283d2d7526f4fc9.wav,фильм пассажир в хорошем качестве,neutral
...,...,...,...
158411,train/895b299e0a21323dffdd0eb0dfa1345c.wav,пгт смирных аэродром,neutral
158412,train/b5174074a9b257251ab0c026bbd581f4.wav,пузырек своими руками,neutral
158413,train/b5174074a9b257251ab0c026bbd581f4.wav,пузырек своими руками,neutral
158414,train/b28cfb2bfc0482cb53c3c9aa09a1df5b.wav,смотреть фильм хэлловин хиби,neutral


In [371]:
test_data = train_frame.iloc[4]

In [372]:
test_data['predicted_text'], test_data['file']

('фильм пассажир в хорошем качестве',
 'train/d3bd2f6258e44b2c1283d2d7526f4fc9.wav')

In [373]:
model(
    test_data['predicted_text'], test_data['file'])

tensor([[-0.4182,  0.0324,  0.2856, -0.4516, -0.5012]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [376]:
optimizer = AdamW(model.parameters(), lr = 5e-5)
loss = nn.CrossEntropyLoss()

In [377]:
loss_array = []

In [386]:
n_epochs = 1

In [384]:
import pickle

In [388]:
with open('dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)

In [392]:
i = 0
for epoch in range(n_epochs):
    for text, audio_path, text_label in tqdm(zip(dataset['predicted_text'], dataset['file'], dataset['label']), total=len(dataset)):
        optimizer.zero_grad()
        label = torch.tensor(emotion2label[text_label]).cuda()
        output = model(text, audio_path)
        loss_ = loss(output, torch.unsqueeze(label, 0))
        loss_array.append(loss_)
        if i % 300 == 0:
            print(loss_)
        loss_.backward()
        optimizer.step()
        i += 1
    

  0%|          | 0/40554 [00:00<?, ?it/s]

tensor(0.0017, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0011, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0007, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0004, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0007, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0006, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(7.0450e-05, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.4397e-05, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.1471e-05, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.1603e-05, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6358e-05, devic

In [400]:
dataset_dataframe = pd.read_csv('test.csv')
dataset_dataframe.columns = ['file', 'label']
dataset_dataframe['emotion'] = dataset_dataframe['label'].replace(label2emotion)
text_frame = pd.read_csv('test_with_text.csv')
text_frame.rename(columns={'audio_path':'file'}, inplace=True)
text_frame.drop(columns=['Unnamed: 0'], inplace=True)
dataset_dataframe = dataset_dataframe.merge(text_frame, how='left', on='file').drop(columns=['emotion'])
dataset_dataframe['predicted_text'].fillna(' ', inplace=True)

In [403]:
model.eval()
true_labels = []
pred_labels = []
i = 0
for text, audio_path, text_label in tqdm(zip(dataset_dataframe['predicted_text'], dataset_dataframe['file'], dataset_dataframe['label']), total=len(dataset_dataframe)):
    label = torch.tensor(emotion2label[text_label]).cpu()
    output = model(text, audio_path)
    pred_labels.append(torch.argmax(output).cpu())
    true_labels.append(label)

  0%|          | 0/24171 [00:00<?, ?it/s]

In [404]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     15886
           1       0.02      0.04      0.02      2481
           2       0.00      0.00      0.00      2506
           3       0.00      0.00      0.00      3072
           4       0.01      0.96      0.02       226

    accuracy                           0.01     24171
   macro avg       0.01      0.20      0.01     24171
weighted avg       0.00      0.01      0.00     24171



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [410]:
true_labels = [int(x) for x in true_labels]
pred_labels = [int(x) for x in pred_labels]

In [411]:
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     15886
           1       0.02      0.04      0.02      2481
           2       0.00      0.00      0.00      2506
           3       0.00      0.00      0.00      3072
           4       0.01      0.96      0.02       226

    accuracy                           0.01     24171
   macro avg       0.01      0.20      0.01     24171
weighted avg       0.00      0.01      0.00     24171



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
